## Remake Duck model in HydroMT

In [ ]:
# import necessary modules
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import path
import time
import xarray as xr
import xugrid as xu

from hydromt_sfincs import SfincsModel
from hydromt_sfincs import utils

from hydromt.log import setuplog

In [ ]:
!hydromt --version

#### Continue with building a QuadTree model

In [ ]:
logger = setuplog("sfincs_duck_hydromt", log_level=10)
sf_qt = SfincsModel(
    data_libs=["deltares_data"], root="test_duck3", mode="w+", logger=logger
)

In [ ]:
# first have reg grid
sf_qt.setup_grid(
    x0=-2.5,
    y0=-2.5,
    dx=5.0,
    dy=5.0,
    nmax=422,
    mmax=179,
    rotation=0,
    epsg=32620,
)

In [ ]:
file_name = r".\input_duck\refine.pol"

refine_gdf = utils.polygon2gdf(feats=utils.read_geoms(fn=file_name), crs=sf_qt.crs)

gdf_refinement = gpd.GeoDataFrame(
    {
        "refinement_level": [
            1,
        ]
    },
    geometry=[
        refine_gdf.unary_union,
    ],
    crs=sf_qt.crs,
)

gdf_refinement

In [ ]:
sf_qt.setup_grid(
    # x0=-2.5,
    # y0=-2.5,
    x0=-5,
    y0=-5,
    dx=10.0,
    dy=10.0,
    nmax=211,
    mmax=90,
    rotation=0,
    epsg=32620,
    refinement_polygons=gdf_refinement,
)

# alternative way of creating a quadtree grid
# sf_qt.setup_grid_from_region(region={"geom": sf.region}, #area that needs to be covered by the grid
#                             res=50, # set resolution
#                             rotated=True, # when True, rotation is determined to minimize the grid extent
#                             refinement_polygons=gdf_refinement)

# NOTE this grid is smaller since the inactive cells of the regular model are already excluded from the grid

Generate topobathy on the quadtree grid

In [ ]:
sf_qt.quadtree.data

In [ ]:
depin = r".\input_duck\delilah.tif"

In [ ]:
sf_qt.quadtree.data["level"].ugrid.plot()

In [ ]:
datasets_dep = [{"elevtn": depin}]

sf_qt.setup_dep(datasets_dep=datasets_dep)

In [ ]:
sf_qt.quadtree.data["dep"].ugrid.plot()

Continue with the mask for the QuadTree grid; we aim to have the same active extent as the regular grid in different ways:
- Based on elevation
- Bu using an include polygon

In [ ]:
# sf_qt.quadtree.setup_mask(include_polygon=mask_include, open_boundary_polygon=open_include, open_boundary_zmax=-3)
sf_qt.quadtree.setup_mask_active(zmin=-999, zmax=999)
# sf_qt.setup_mask_active(zmin=-999, zmax=999)

In [ ]:
# NOTE we lose performance through the SfincsModel, I expect this to come from the data_catalog that parses the geodataframes
# sf_qt.quadtree.setup_mask_active(gdf_include=mask_include, all_touched=False)

In [ ]:
sf_qt.crs

In [ ]:
file_name = r"./input_duck/bnd_wl_delilah_Left.pol"

open_include = utils.polygon2gdf(feats=utils.read_geoms(fn=file_name), crs=sf_qt.crs)

In [ ]:
sf_qt.setup_mask_bounds(btype="waterlevel", zmax=0, include_mask=open_include)  #

# sf_qt.grid['msk'].plot(x="x", y="y")
sf_qt.quadtree.data["msk"].ugrid.plot()

In [ ]:
# plot the difference between the mask (original code) and the msk (new code)
# NOTE with all_touched=True, there is a small difference
# (sf_qt.quadtree.data["mask"]-sf_qt.quadtree.data["msk"]).ugrid.plot()

In [ ]:
# drop the mask variable from the quadtree data
# sf_qt.quadtree.data = sf_qt.quadtree.data.drop("mask")

## Now some snapwave functionalities

In [ ]:
# sf_qt.setup_mask_active(model="snapwave", zmin=-999, zmax=2)#, include_mask=gdf_riv_buf)
# sf_qt.quadtree.data["snapwave_msk"].ugrid.plot()
sf_qt.setup_mask_active(model="snapwave", zmin=-999, zmax=0)
sf_qt.quadtree.data["snapwave_msk"].ugrid.plot()

Try situation where we want to directly copy the SFINCS mask to SnapWave:

In [ ]:
# sf_qt.setup_mask_bounds(model="snapwave", copy_sfincsmask=True)
# sf_qt.quadtree.data["snapwave_msk"].ugrid.plot()

### Adjust offshore wave boundary for SnapWave:

In [ ]:
# %matplotlib qt
%matplotlib inline

sf_qt.setup_mask_bounds(
    model="snapwave", include_mask=open_include, copy_sfincsmask=False
)
fig = sf_qt.quadtree.data["snapwave_msk"].ugrid.plot()
# fig.aspect('equal', adjustable='box')

In [ ]:
sf_qt.quadtree.data

### Now create a subgrid table for this model

In [ ]:
# sf_qt.setup_subgrid(datasets_dep=datasets_dep, buffer_cells=40)

### Add some random boundary conditions

In [ ]:
sf_qt.config["tref"] = "19901013 160000"
sf_qt.config["tstart"] = "19901013 160000"
sf_qt.config["tstop"] = "19901013 170000"

In [ ]:
# x&y-locations in same coordinate reference system as the grid:
x = [0, 0]
y = [0, 2100]

# add to Geopandas dataframe as needed by HydroMT
pnts = gpd.points_from_xy(x, y)
index = [1, 2]  # NOTE that the index should start at one
bnd = gpd.GeoDataFrame(index=index, geometry=pnts, crs=sf_qt.crs)

# In this case we will provide 3 values (periods=3) between the start (tstart=20100201 000000) and the end (tstop=20100201 120000) of the simulation:
time = pd.date_range(
    start=utils.parse_datetime(sf_qt.config["tstart"]),
    end=utils.parse_datetime(sf_qt.config["tstop"]),
    periods=2,
)

# add some water levels
bzs = [[0.69, 0.69], [0.69, 0.69]]

bzspd = pd.DataFrame(index=time, columns=index, data=bzs)

# Actually add it to the SFINCS model class:
sf_qt.setup_waterlevel_forcing(timeseries=bzspd, locations=bnd)

### Add wave boundary conditions:

In [ ]:
# Wanted 2 locations:
gdf = utils.read_xy("input_duck//delilah_01.bnd", crs=sf_qt.crs)

# Wanted values:
hs = [[1.81, 1.81], [1.81, 1.81]]
tp = [[10.6, 10.6], [10.6, 10.6]]
dir = [[286.0, 286.0], [286.0, 286.0]]
ds = [[30.6, 30.6], [30.6, 30.6]]
time = [0, 99999]

df_hs = pd.DataFrame(index=time, data=hs)
df_tp = pd.DataFrame(index=time, data=tp)
df_dir = pd.DataFrame(index=time, data=dir)
df_ds = pd.DataFrame(index=time, data=ds)

list_df = [df_hs, df_tp, df_dir, df_ds]

sf_qt.setup_wave_forcing(timeseries=list_df, locations=gdf)

gdf

In [ ]:
sf_qt.plot_forcing()

### Add wavemaker:

!NOTE - Clicking order should be anti-clockwise to force a wave to the left!!!

In [ ]:
wvm = sf_qt.setup_wavemaker(wavemaker="input_duck//wavemaker.pol", merge=False)

sf_qt.geoms.keys()

In [ ]:
sf_qt.geoms["wvm"]

### And save everything we build sofar

In [ ]:
sf_qt.config["tspinup"] = 60
sf_qt.config["alpha"] = 0.5
sf_qt.config["theta"] = 1.0

sf_qt.config["advection"] = 1

sf_qt.config["zsini"] = 0.69
sf_qt.config["dtout"] = 10.0

sf_qt.config["obsfile"] = "../input_duck/delilah_points_ext.obs"

In [ ]:
# SnapWave specific settings:
sf_qt.config["dtwave"] = 600
sf_qt.config["storefw"] = 1
sf_qt.config["storewavdir"] = 1
sf_qt.config["snapwave"] = 1
sf_qt.config["snapwave_igwaves"] = 1

# sf_qt.config["snapwave_bndfile"]        = "../input_duck/delilah_01.bnd"
# sf_qt.config["snapwave_bhsfile"]        = "../input_duck/delilah_real.bhs"
# sf_qt.config["snapwave_btpfile"]        = "../input_duck/delilah_real.btp"
# sf_qt.config["snapwave_bwdfile"]        = "../input_duck/delilah_real.bwd"
# sf_qt.config["snapwave_bdsfile"]        = "../input_duck/delilah_real.bds"

sf_qt.config["snapwave_shinc2ig"] = 1.0
sf_qt.config["snapwave_fw"] = 0.0001
sf_qt.config["snapwave_fwig"] = 0.015
sf_qt.config["snapwave_baldock_opt"] = 1
sf_qt.config["snapwave_baldock_ratio"] = 0.2
sf_qt.config["snapwave_baldock_ratio_ig"] = 0.2
sf_qt.config["snapwave_alpha"] = 1.5
sf_qt.config["snapwave_alpha_ig"] = 2.5
sf_qt.config["snapwave_gamma"] = 0.78
sf_qt.config["snapwave_gammaig"] = 0.2
sf_qt.config["snapwave_ig_opt"] = 1
sf_qt.config["snapwave_hmin"] = 0.01
sf_qt.config["snapwave_shpercig"] = 1.0
sf_qt.config["snapwave_dtheta"] = 10
sf_qt.config["snapwave_alphaigfac"] = 1.0
sf_qt.config["snapwave_use_herbers"] = 1

In [ ]:
sf_qt.write()

### Run simulation:

In [ ]:
# run_path = "./test_duck3"

# cur_dir = os.getcwd()

# # uncomment to run sfincs
# os.chdir(run_path)
# os.system("run_sfincs.bat")
# os.chdir(cur_dir)

### Read results, make plots and animation:

In [ ]:
import xugrid as xu

uds = xu.open_dataset(r".\test_duck3\sfincs_map.nc")

uds

In [ ]:
uds["zb"].ugrid.plot()

In [ ]:
uds["snapwavedepth"][-1, :].ugrid.plot()

In [ ]:
uds["msk"].ugrid.plot()

In [ ]:
uds["snapwavemsk"].ugrid.plot()

In [ ]:
uds["zsmax"].max(dim="timemax").ugrid.plot(vmax=1.0)

In [ ]:
zsmax = uds["zsmax"].max(dim="timemax")
# plot rough estimation of water depth
h = zsmax - uds["zb"]

zsmax = zsmax[h > 0.1]

zsmax.ugrid.plot(vmax=2.0)

In [ ]:
uds["zsm"][-1, :].ugrid.plot(vmax=5.0)

In [ ]:
zsm = uds["zsm"][-1, :]
# plot rough estimation of water depth
h = zsm - uds["zb"]

zsm = zsm[h > 0.1]

zsm.ugrid.plot(vmax=1.0)

In [ ]:
da = uds["zb"].ugrid.sel(y=1050)
da.plot(x="mesh2d_s", label="zb")

da = uds["zsm"][-1, :].ugrid.sel(y=1050)
da.plot(x="mesh2d_s", label="zs mean")

da = uds["zs"][-1, :].ugrid.sel(y=1050)
da.plot(x="mesh2d_s", label="zs")

da = uds["zsmax"].max(dim="timemax").ugrid.sel(y=1050)
da.plot(x="mesh2d_s", label="zs max")

plt.legend()
plt.xlim(600)
plt.ylim(-5, 5)

In [ ]:
uds["hm0"][-1, :].ugrid.plot(vmax=1.5 * np.sqrt(2))

In [ ]:
da = uds["hm0"][-1, :].ugrid.sel(y=1050) / np.sqrt(2)
da.plot(x="mesh2d_s", label="incident")

da = uds["hm0ig"][-1, :].ugrid.sel(y=1050) / np.sqrt(2)
da.plot(x="mesh2d_s", label="IG")

plt.legend()

In [ ]:
uds["hm0ig"][-1, :].ugrid.plot(vmax=0.35 * np.sqrt(2))

In [ ]:
# plot rough estimation of water depth
h = uds["zsmax"].max(dim="timemax") - uds["zb"]

h = h[h > 0.1]

h.ugrid.plot()

### 2D map animation:

In [ ]:
# # create zs plot and save to mod.root/figs/sfincs_zs.mp4
# # requires ffmpeg install with "conda install ffmpeg -c conda-forge"
# import matplotlib.pyplot as plt
# import numpy as np
# from matplotlib import animation

# step = 1  # one frame every <step> dtout
# cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
# da_zs = uds["zs"]

# def update_plot(i, da_zs, cax_zs):
#     da_zsi = da_zs.isel(time=i)
#     t = da_zsi.time.dt.strftime("%d-%B-%Y %H:%M:%S").item()
#     ax.set_title(f"SFINCS water level {t}")
#     cax_zs.set_array(da_zsi.values.ravel())

# fig, ax = plt.subplots(figsize=(11, 7))
# cax_zs = da_zs.isel(time=0).ugrid.plot(
#     ax=ax, vmin=0, vmax=1, cmap=plt.cm.viridis, cbar_kwargs=cbar_kwargs
# )
# plt.close()  # to prevent double plot

# ani = animation.FuncAnimation(
#     fig,
#     update_plot,
#     frames=np.arange(0, da_zs.time.size, step),
#     interval=250,  # ms between frames
#     fargs=(
#         da_zs,
#         cax_zs,
#     ),
# )

# # to save to mp4
# ani.save(os.path.join(sf_qt.root, 'sfincs_duck_zs.mp4'), fps=4, dpi=200)

# # to show in notebook:
# from IPython.display import HTML

# HTML(ani.to_html5_video())

### 1D transect animation:

In [ ]:
# create zs plot and save to mod.root/figs/sfincs_zs.mp4
# requires ffmpeg install with "conda install ffmpeg -c conda-forge"
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation

step = 5  # 1  # one frame every <step> dtout
cbar_kwargs = {"shrink": 0.6, "anchor": (0, 0)}
time_zs = uds["time"]
da_zs = uds["zs"].ugrid.sel(y=1050)
da_zb = uds["zb"].ugrid.sel(y=1050)
da_zsm = uds["zsm"].ugrid.sel(y=1050)
da_zsmax = uds["zsmax"].max(dim="timemax").ugrid.sel(y=1050)


def update_plot(i, da_zs, cax_zs):
    ax.clear()

    da_zsi = da_zs.isel(time=i)
    # t = da_zsi.time.dt.strftime("%d-%B-%Y %H:%M:%S").item()
    # t = time_zs[i]
    t = da_zsi.time.dt.strftime("%d-%B-%Y %H:%M:%S").item()
    cax_zs = da_zs.isel(time=i).plot(ax=ax, x="mesh2d_s", label="zs")
    cax_zsm = da_zsm.isel(time=i).plot(ax=ax, x="mesh2d_s", label="zsm")
    cax_zb = da_zb.plot(ax=ax, x="mesh2d_s", label="zb")
    cax_zsmax = da_zsmax.plot(ax=ax, x="mesh2d_s", label="zs max")
    # plt.legend()
    ax.set_title(f"SFINCS water level {t}")
    ax.set_xlim(600)
    ax.set_ylim(-5, 5)
    ax.legend()
    # cax_zs.set_array(da_zsi.values.ravel())
    # cax_zs = da_zs[i,:].plot(ax=ax)
    # cax_zs.set_data(da_zsi.values.ravel())

    # cax_zs.set_array
    # plt.close()


fig, ax = plt.subplots(figsize=(11, 7))
# cax_zs = da_zs[0,:].ugrid.sel(y=1050).plot(ax=ax, label='zs')
# cax_zs = da_zs.isel(time=0).plot(ax=ax, label='zs')
# cax_zb = da_zb.plot(ax=ax, label='zb')
# cax_zs = da_zs[0,:].ugrid.sel(y=1050).plot(ax=ax, label='zs')
# cax_zsmax = da_zsmax.plot(ax=ax, label='zs max')

plt.close()  # to prevent double plot

ani = animation.FuncAnimation(
    fig,
    update_plot,
    frames=np.arange(0, da_zs.time.size, step),
    interval=250,  # ms between frames
    fargs=(
        da_zs,
        cax_zs,
    ),
)

# to save to mp4
ani.save(os.path.join(sf_qt.root, "sfincs_duck_zs_1D.mp4"), fps=4, dpi=200)

# to show in notebook:
from IPython.display import HTML

HTML(ani.to_html5_video())

In [ ]:
uds["msk"].where(uds["msk"] == 2, np.nan).ugrid.plot()